# Project 2 : :)
## Authors:

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

In [ ]:
## IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import loader.py
import plotting.py
import spirals.py

## GLOBAL CONSTS AND DEFINITIONS
sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
sech = lambda x : 1/np.cosh(x)
eta_ddx = lambda x :  0.25*sech(x/2)*sech(x/2) # Derivative of eta
sigma_ddx = lambda x: sech(x)*sech(x) # Derivative of sigma
h = 0.1                               # Step length          
#_____________________________________________________________

In [ ]:
J = 1/2*np.linalg.norm(Z-c)**2 # Cost function
#But Z and C are not defined

In [ ]:
## Adam descent algorithn
def AdamAlg():
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    v[0] = 0
    m[0] = 0
    k =3 #Debug
    for j in range(1,k):
        g[j] = uGrad(J(U[j]))
        m[j] = beta1*m[j-1]+(1-beta1)*g[j]
        v[j] = beta2*v[j-1]+(1-beta2)(g[j]*g[j])
        mhat[j] = m[j]/(1-beta1**j)
        vhat[j] = v[j]/(1-beta2**j)
        U[j+1] -= alpha*mhat[j]/(np.sqrt(vhat[j])+e)
def getGradients():
    P[K] = omega*np.transpose((Z-c)*eta_ddx(np.transpose(Y[K])*omega+mu*One))
    dJdMU = eta_ddx(np.transpose(np.transpose(Y[K])*omega+mu*One)*(Z-c))
    dJdOmega = Y[K]*((Z-c)*eta_ddx(np.transpose(Y[K])*omega+mu))
    for k in range(K,-1,2):
        P[k-1] = P[k]+h*np.transpose(W[k-1])*(sigma_ddx(W[k-1]*Y[k-1]+b[k-1])*P[k])
    for k in range(0,K-1):
        dJdWk[k] = h(P[k+1]*sigma_ddx(W[k]*Y[k]+b[k]))*np.transpose(Y[k])
        dJdBk[k] = h(P[k+1]*sigma_ddx(W[k]*Y[k]+b[k]))*One 
### THINGS IM UNCERTAIN ABOUT
# Matrix mult @ or * for eq (9) and (10)??
# First for-loop in getGradients, is the range correct?
# Parameters for Adam alg?
# What is Y[k]?
# How to readData()?
# How to check if converged? And when?

In [ ]:
# Algorithm
K = 4 # Amount of layers, arbitrary number
d = 3 # Ehh
tau = 0.07 # learning parameter [0.01,0.1]
Y0 = readData() # To be implemented
Wk, bk = np.random.randn(k,d,d), np.random.randn(k,d,1)
converged = False
while not converged:
    for k in range(1,K):
        #Calculate Y[k] and save to memory
    getGradients()
    AdamAlg()
    converged = True #Only one iteration when debugging